<a href="https://colab.research.google.com/github/polizzilab/design_tools/blob/main/loop_inpainting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##ColabPaint: Inpainting using RFDesign

Easy to use protein looping / inpainting using [RFDesign](https://www.science.org/doi/abs/10.1126/science.abn2100?af=R). 

In [ ]:
#@title Input protein "contigs" and loop lengths, then hit `Runtime` -> `Run all`
import os
from google.colab import files
import hashlib
import re

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

jobname = 'test' #@param {type:"string"}
# rbdbinder_6vw1
jobname.replace('.', '-')
# remove whitespaces
basejobname = "".join(jobname.split())
basejobname = re.sub(r'\W+', '', basejobname)
jobname = add_hash(basejobname, basejobname)

contigs_and_loop_lengths = 'A1-27, 3-7, A33-60, 3-7, A67-93, 3-7, A101-126' #@param {type:"string"}
#@markdown *contigs_and_loop_lengths*: comma delimited list of residues (starting with chain letter) and inpainted residue lengths (starting with an integer). See **Notes about the protocol** below for more details. 
#loop_lengths = '0,70' #@param {type:"string"}
# contigs='A1-45,3-9,A55-75,4-8,A84-126'
# E334-521, 0, A24-41, 40
do_global_loop_search = True #@param {type:"boolean"}
only_output_top_local_loops = False #@param {type:"boolean"}
num_cycles = 15 #@param {type: "integer"}
tmpl_conf = 0.75 #@param {type: "number"}

os.chdir('/content/')

contigs_and_loop_lengths = contigs_and_loop_lengths.replace(' ','').split(',')
#loop_ranges = loop_lengths.replace(' ','').split(',')

#@markdown You will be prompted to upload a PDB file to continue.
custom_template_path = f"{jobname}_template/"
if not os.path.exists(custom_template_path):
  os.mkdir(custom_template_path)
uploaded = files.upload()
for fn in uploaded.keys():
  # remove any preceding '.'s in filename
  count = fn.count(".") - 1
  newfn = fn.replace('.', '-', count)
  newfn = fn.replace('_', '-')
  os.rename(fn, f"{jobname}_template/{newfn}")
pdbname = newfn.split('.')[0]



In [ ]:
#@title Process local contigs
from collections import defaultdict
import itertools
element_dict = defaultdict(list)
prev_elem_type = ''
elem_num = 0
for elem in contigs_and_loop_lengths:
  try:
    int(elem[0])
    elem_type = int
  except:
    elem_type = str
  if elem_type != prev_elem_type:
    elem_num += 1
  element_dict[elem_num].append(elem)
  prev_elem_type = elem_type

constant_keys = []
keys = sorted(element_dict.keys())
for key in keys:
  try:
    int(element_dict[key][0][0])
    if len(element_dict[key]) == 1 and '-' not in element_dict[key][0]:
      constant_keys.append(key)
  except:
    pass
all_constant_keys = {j for i in constant_keys for j in [i-1, i, i+1] if j in keys}

contigs = []
variable_keys = sorted(set(keys) - all_constant_keys)
for i, vkey in enumerate(variable_keys):
  if not element_dict[vkey][0][0].isnumeric():
    continue
  printed_keys = sorted((all_constant_keys | {vkey - 1, vkey, vkey + 1}) & set(keys))
  iterable = element_dict[vkey]
  variable_index = printed_keys.index(vkey)

  processed_iterable = []
  for it in iterable:
    if '-' in it:
      loop_start, loop_stop = it.split('-')
      loop_start = int(loop_start)
      loop_stop = int(loop_stop)
      for j in range(loop_start, loop_stop + 1):
        processed_iterable.append(str(j))
    else:
      processed_iterable.append(it)

  contig = []
  for key in printed_keys:
    if key == vkey:
      contig.append('')
    else:
      contig.append(element_dict[key][0])

  for pit in processed_iterable:
    contig[variable_index] = pit
    contigs.append((','.join(contig), str(i)))
# contigs
if len(variable_keys) == 0:
  contig = []
  for key in sorted(all_constant_keys):
    contig.append(element_dict[key][0])
  contigs.append((','.join(contig), str(i)))



In [ ]:
#@title Make job files
def make_job_file(contig_str, tag, outdir):
  if outdir[-1] != '/':
    outdir += '/'
  outname_suffix = '_'.join(contig_str.split(',')) + '_inpaint' + tag
  outname = outdir + pdbname + '_' + outname_suffix

  with open(f'{outname}.sh', 'w') as jobfile:
    jobfile.write('#!/bin/bash \n')
    pdbpath = custom_template_path + pdbname
    jobfile.write(f"pdb='{pdbpath}.pdb' \n")
    jobfile.write(f"contigs='{contig_str}' \n")
    jobfile.write(f"out='{outname}' \n")
    jobfile.write('num_designs=1 \n')
    jobfile.write(f'tmpl_conf={tmpl_conf} \n')
    script = "script='/content/RFDesign/inpainting/inpaint.py' \n"
    jobfile.write(script)
    jobfile.write('mkdir -p `dirname $out` \n')
    jobfile.write(f'python $script --pdb $pdb --contigs $contigs --dump_trb --dump_pdb --n_cycle {num_cycles} --out $out --num_designs $num_designs --tmpl_conf $tmpl_conf \n')

local_loop_path = custom_template_path + 'local_loops/'
if not os.path.exists(local_loop_path):
  os.mkdir(local_loop_path)

for contig, tag in contigs:
    make_job_file(contig, tag, local_loop_path)

In [ ]:
#@title Make minimal environment file

with open('env.yml', 'w') as envfile:
  envfile.write(
      """
  name: SE3-nvidia \n
  channels: \n
    - rusty1s \n
    - pytorch \n
    - dglteam \n
    - nvidia \n
    - conda-forge \n
    - defaults \n
  dependencies: \n
    - cudatoolkit=11.1.74=h6bb024c_0 \n
    - numpy=1.20.3=py39hf144106_0 \n
    - numpy-base=1.20.3=py39h74d4b33_0 \n
    - pip=21.2.4=pyhd8ed1ab_0 \n
    - dgl-cuda11.1=0.8.2=py39_0 \n
    - psutil=5.9.0=py39hb9d737c_1 \n
    - python=3.9.6=h49503c6_1_cpython \n
    - pytorch=1.9.0=py3.9_cuda11.1_cudnn8.0.5_0 \n
    - pytorch-cluster=1.5.9=py39_torch_1.9.0_cu111 \n
    - pytorch-geometric=1.7.2=py39_torch_1.9.0_cu111 \n
    - pytorch-scatter=2.0.8=py39_torch_1.9.0_cu111 \n
    - pytorch-sparse=0.6.11=py39_torch_1.9.0_cu111 \n
    - pytorch-spline-conv=1.2.1=py39_torch_1.9.0_cu111 \n
    - requests=2.26.0=pyhd8ed1ab_0 \n
    - requests-unixsocket=0.2.0=py_0 \n
    - scikit-learn=0.24.2=py39ha9443f7_0 \n
    - scipy=1.6.2=py39had2a1c9_1 \n
    - pip: \n
      - icecream==2.1.1 \n
      - lie-learn==0.0.1.post1 \n
      - opt-einsum==3.3.0 \n
      - e3nn==0.3.4 \n
  prefix: ./SE3-nvidia \n
  """
  )

In [ ]:
#@title Install dependencies
#@markdown This may take 10-15 min but only needs to be done once per session. 
#@markdown The notebook will skip over this part if run again. 
%%bash -s 

set -e

if [ ! -f RFDES_READY ]; then
  echo "Cloning RFDesign..."
  git clone https://github.com/RosettaCommons/RFDesign.git
  touch RFDES_READY
fi

# setup conda
if [ ! -f CONDA_READY ]; then
  echo "Installing Conda..."
  wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
  bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
  rm Miniconda3-latest-Linux-x86_64.sh
  touch CONDA_READY
fi

if [ ! -f ENV_READY ]; then
  echo "Creating Conda SE3-nvidia environment for RFDesign"
  conda env create -f /content/env.yml 2>&1 1>/dev/null
  touch ENV_READY
fi

if [ ! -f BFF_WEIGHTS ]; then
  echo "Downloading BFF_mix_epoch25 weights..."
  wget http://files.ipd.uw.edu/pub/rfdesign/weights/BFF_mix_epoch25.pt >> /dev/null 2>&1  
  mkdir /content/RFDesign/inpainting/weights/
  mv /content/BFF_mix_epoch25.pt /content/RFDesign/inpainting/weights/BFF_mix_epoch25.pt
  touch BFF_WEIGHTS
fi


In [ ]:
#@title Run inpainting tests
#@markdown Test to see if installation was successful. Should end with "Successfully wrote output".
%%bash -s

if [ ! -f TEST_READY ]; then
  echo "Running inpainting tests..."
  source activate SE3-nvidia
  cd /content/RFDesign/inpainting/tests/
  conda init bash >> /dev/null 2>&1
  conda activate SE3-nvidia
  ./run_tests.sh
  touch /content/TEST_READY
fi

In [ ]:
#@title Run local loop inpainting
%%bash -s $local_loop_path

LOCAL_LOOP_PATH=$1

source activate SE3-nvidia 
conda init bash >> /dev/null 2>&1
conda activate SE3-nvidia

for file in $LOCAL_LOOP_PATH*.sh; do
    chmod +x $file
    $file 
done 
echo "Done with local inpainting."


In [ ]:
#@title Get top N inpainted local loops by mean LDDT
topN = 3 #@param {type:"integer"}
import numpy as np
from collections import defaultdict
loop_dict = defaultdict(list)

inpaint_group_dict = defaultdict(list)
for trb_file in os.listdir(local_loop_path):
  if trb_file.split('.')[-1] != 'trb':
    continue
  trb_spl = trb_file.split('.')[0].split('_')
  inpaint_group = trb_spl[-2].split('inpaint')[-1]
  trb = np.load(local_loop_path + trb_file, allow_pickle=True)
  mean_lddt = np.array(trb['inpaint_lddt']).mean()
  inpaint_group_dict[inpaint_group].append((mean_lddt, tuple(trb_spl)))

sorted_inpaint_group_dict = defaultdict(set)
best_inpaint_group_dict = dict()
for inpaint_group, mean_lddts in inpaint_group_dict.items():
  sorted_trbs = sorted(mean_lddts, reverse=True)[:topN]
  best_inpaint_group_dict[inpaint_group] = [sorted_trbs[0][-1]]
  for trb in sorted_trbs:
    sorted_inpaint_group_dict[inpaint_group].add(trb[-1])


In [ ]:
#@title Make global loop-finding job files
def combine_to_single_contig(*trbs):
  contig = []
  for trb in trbs:
    trb = trb[1:-2]
    for i in range(0, len(trb), 2):
      try:
        bigram = trb[i] + "," + trb[i + 1]
      except:
        bigram = trb[i]
      if bigram in contig:
        continue
      contig.append(bigram)
  return ','.join(contig)

if do_global_loop_search:
  glob_dict = sorted_inpaint_group_dict
else:
  glob_dict = best_inpaint_group_dict

glob_contigs = []
for trbs in itertools.product(*[glob_dict[key] for key in sorted(glob_dict.keys())]):
  glob_contigs.append(combine_to_single_contig(*trbs))

global_loop_path = custom_template_path + 'global_loops/'
if not os.path.exists(global_loop_path):
  os.mkdir(global_loop_path)

if not only_output_top_local_loops:
  for glob_contig_str in glob_contigs:
    make_job_file(glob_contig_str, 'global', global_loop_path)

In [ ]:
#@title Run global loop inpainting
%%bash -s $global_loop_path

GLOBAL_LOOP_PATH=$1

source activate SE3-nvidia 
conda init bash >> /dev/null 2>&1
conda activate SE3-nvidia

for file in $GLOBAL_LOOP_PATH*.sh; do
    chmod +x $file
    $file 
done
echo "Done with global inpainting."



In [ ]:
#@title Get top N inpainted global loops by mean LDDT
topN = 3 #@param {type:"integer"}

if only_output_top_local_loops:
  loop_path = local_loop_path
else:
  loop_path = global_loop_path

mean_lddts = []
for trb_file in os.listdir(loop_path):
  if trb_file.split('.')[-1] != 'trb':
    continue
  trb = np.load(loop_path + trb_file, allow_pickle=True)
  mean_lddt = np.array(trb['inpaint_lddt']).mean()
  mean_lddts.append((mean_lddt, trb_file))

sorted_loops = sorted(mean_lddts, reverse=True)[:topN]

output_loop_path = custom_template_path + 'output/'
if not os.path.exists(output_loop_path):
  os.mkdir(output_loop_path)

import shutil
for i, loopfile in enumerate(sorted_loops):
  pdbname_ = loopfile[-1].split('.')[0]
  pdb = pdbname_ + '.pdb'
  pdbpath = loop_path + pdb
  dst = output_loop_path + pdbname_ + '_rank_' + str(i + 1) + '_' + str(np.round(loopfile[0], 4)) + '.pdb'
  shutil.copyfile(pdbpath, dst)


In [ ]:
#@title Package and download results
#@markdown If you are having issues downloading the result archive, try disabling your adblocker and run this cell again. If that fails click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\" (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).
!cp $global_loop_path"FLAGS.txt" $custom_template_path"output/"
!zip -FSr $jobname".result.zip" $custom_template_path"output"
files.download(f"{jobname}.result.zip")


# Instructions <a name="Instructions"></a>
**Quick start**
1. Paste your protein sequence(s) in the input field. Uncheck "do_global_loop_search" if you want the best local loop solution only (faster).
2. Press "Runtime" -> "Run all".

**Result zip file contents**

1. PDB formatted structures sorted by avg. pLDDT of loops. File names include the rank (lower is better) and avg. pLDDT (higher is better), averaged over all inpainted loops. 

2. FLAGS.txt file with input parameters from the global loop search.

At the end of the job a download modal box will pop up with a `jobname.result.zip` file. Note that this protocol is not extensively tested. It might break for your protein. Use at own risk!

**Notes about the protocol**

- From my limited playing around with the code, I found that searching for all  loops at once gave different answers (at least wrt sequence) than searching for loops one at a time. Doing "local" loop inpainting should provide information on what loop lengths are feasible, which we can use to avoid combinatorial explosion of loop lengths. For example, if we had 3 loops and wanted to vary loop length from 3-13 for each loop, that would be 10 * 10 * 10 combinations. Some of those loop lengths won't be good “local” solutions anyway (low avg. LDDT), so we can discard them. But first we must compute the “local loop solutions” in order to rank them. Then we can take the top N from each local inpainting (N=3 is default); so instead of 10 * 10 * 10 combos we have N * N * N. If N = 2 or 3 then the computation is much more tractable. 
- One could also choose to ignore these combinatorics altogether by deselecting `do_global_loop_search` at the top of the notebook, which will result in outputing the best "local loop solution" for each loop by setting the loop lengths of one inpainting job to the best local loop lengths.
- If you want a chain to be present for context but don't want to add a loop to that chain, use a loop length of 0. For example, for inpainting an extension of the ACE2 helix in the binding interface of Sars-CoV-2 RBD (pdb 6wv1), a *contigs_and_loop_lengths* input might be "E334-521, 0, A24-41, 40,50,60". This will extend A24-41 (the ACE2 helix) by 40, 50, and 60 residues in the context of chain E residues 334-521 (the RBD). If one wanted to prepend these residues to A24, the appropriate *contigs_and_loop_lengths* input might be "40,50,60, A24-41, 0, E334-521".
- Loop lengths can be denoted as a comma delineated list following a stretch of residue numbers, e.g., "A21-42,10,15,20" will inpaint loops of length 10, 15, and 20 after residue 42 of chain A; "B2-100,2,5,7-10" will do inpaint loops of length of 2,5,7,8,9, and 10 after residue 100 of chain B.
-Inpainting can be used for extending a protein or prepending a protein, as well as adding loops.
-Increase the number of cycles (num_cycles) if inpainting gives low pLDDT solutions. Perhaps a good heuristic is num_cycles >= 2 * longest_loop_length.
-Check "only_output_top_local_loops" if you want to skip the global inpainting entirely and just want the best local loops outputted. This is would be preferred (faster) if you are just inpainting one loop or only exploring a single loop length for multiple loops.
-Increase *tmpl_conf* (range 0-1) if you want the rmsd between the output and your input to be lower.